# **P**redicting **A**ccurate **S**pectral **T**races for **A**strophysical **SOSS** Spectra Package Demo 
----

This notebook demonstrates how to use the `PASTASOSS` package to rapidly generate the trace positions for NIRISS/SOSS observations, in addition we include and example SOSS observation. `PASTASOSS` uses reference models of trace positions for commanded position of the GR700XD, for order 1 and 2. We apply a rotation transform to reference models to derive the trace positions for any given pupil wheel position angle that is included in the FITS header using the keyword `PWCPOS`. The associate wavelength for each trace positions will be added to the tool at a later date to provide a complete picture with sub-pixel performance. Future iterations of this will include update models for spectral traces positions and will eventually include support for order 3. 

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pastasoss

## Generate Trace Positions

First's, load in an example image that has been provided for this notebook and the pupil wheel position associated with the SOSS observation has been given. We will demonstratte how to use `PASTASOSS` to generate the trace positions for this observation given the pupil wheel position.

The example SOSS observation used in this demo is from the calibration program 1512 dataset. 

Let's begin by loading in the image for this demo:

In [ ]:
# Load the .npy file containing only the image.
data = np.load('jw01512001001_03102_00001_nis_demo_pwcpos_245.778671.npy')

# Pupil wheel position assocaitated with the example observation. 
pwcpos = 245.778671

# Plot the image
plt.figure(figsize=(6,3), dpi=200)
plt.imshow(data, vmin=0, vmax=50, origin='lower', aspect='auto',cmap='inferno')
plt.xlabel('x [pix]')
plt.ylabel('y [pix]')
plt.show()


Next, will we generate the trace positions for spectral orders 1 and 2 using the `get_soss_traces` function from `PASTASOSS`.

In [ ]:
# get the order 1 traces for the desired PWCPOS 
traces_order1 = pastasoss.get_soss_traces(pwcpos=pwcpos, order='1', interp=True) 

# now for order 2
traces_order2 = pastasoss.get_soss_traces(pwcpos=pwcpos, order='2', interp=True)

The `get_soss_traces` method will use the included trace and wavelength calibration model to predict the trace (x, y) pixel positions and their associated wavelength values in units of microns. This method will return a TraceModel that is a dataclass object to store the trace properties (i.e., order, x, y, wavelength). 

In [ ]:
type(traces_order1)

In [ ]:
print(traces_order1)

You can also called the function in a single line by the follow:

In [ ]:
# uncomment this line and run the cell
# traces_order1, traces_order2 = pastasoss.get_trace_from_reference_transform(pwcpos=pwcpos, order='12', interp=True)

Next, lets plot our traces we just generated for this observation ontop of our image. Let separate our traces into their x-,y-compenents.

In [ ]:
x1, y1, wave1 = traces_order1.x, traces_order1.y, traces_order1.wavelength
x2, y2, wave2 = traces_order2.x, traces_order2.y, traces_order2.wavelength

Now lets plots traces for orders 1 and 2 on top of our example SOSS observations

In [ ]:
plt.figure(figsize=(6,3), dpi=200)
plt.title(f'NIRISS/SOSS GR700XD\ntrace positions at PWCPOS={pwcpos:.3f}')
plt.imshow(data, vmin=0, vmax=50, origin='lower', aspect='auto', cmap='inferno')
plt.plot(x1,y1, lw=1.5, label=f'order 1', color='cornflowerblue')
plt.plot(x2,y2, lw=1.5, label=f'order 2', color='orangered')
plt.xlabel('x [pix]')
plt.ylabel('y [pix]')
plt.legend()
plt.show()

Now that we have our traces positions for spectral order 1 and 2, we can perform a simple aperutre extraction to extract the spectrum.  

In [ ]:
# ignore values less zero
data[data<0] = 0

# Define how many pixels we want to set our aperture above and below the trace center
npix = 15

# perform a simple aperture extraction via cutout of a desired window size. 
flux_order1 = [data[int(y)-npix:int(y)+npix, int(x)].sum() for x, y in zip(x1, y1)]
flux_order2 = [data[int(y)-npix:int(y)+npix, int(x)].sum() for x, y in zip(x2, y2)]

Let's plot the extracted spectrum for orders 1 and 2.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
fig.suptitle('Extracted SOSS Spectra')
ax1.set_title('Order 1')
ax1.plot(wave1, flux_order1, lw=1.5, label=f'order 1', color='cornflowerblue')
ax1.set_xlabel('wavelength [um]')
ax1.set_ylabel('DN/s')


ax2.set_title('Order 2')
ax2.plot(wave2, flux_order2, lw=1.5, label=f'order 2', color='orangered')
ax2.set_xlabel('wavelength [um]')
ax2.set_ylabel('DN/s')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,6), dpi=187)
plt.plot(wave1, flux_order1, lw=1.5, label=f'order 1', color='cornflowerblue')
plt.xlabel('wavelength [um]')
plt.ylabel('DN/s')
plt.text(1.25, 25000, 'Order 1', color='cornflowerblue')
plt.twiny()
plt.plot(wave2, flux_order2, lw=1.5, label=f'order 2', color='orangered')
plt.text(0.7, 20000, 'Order 2', color='orangered')
plt.ylabel('DN/s (order 2)')
plt.xlabel('wavelength [um]')
# plt.legend()

plt.tight_layout()
plt.show()

# Conclusion

This concludes the demo (v1.1) of how to use the PASATSOSS Package.

Our goal with PASTASOSS to provide the community with a tool to predict the spectral traces (i.e, their positions on the detector and associated wavelengths for the three GR700XD diffraction orders) given a PWCPOS value. 

### Future priority updates to include into the PASTASOSS package:
1. Support for order 3 traces and wavelength calibration
2. Update trace and wavelength calibration models when more data becomes available
3. Possible integration into the JWST calibration pipeline (TBD). 


### Additional features that may be included in the future:
1. integrated method(s) to trace spectral such as:
    - the applesoss edge-triger algorithm 
    - transitspectroscopy cross-correlation algorithm
2. spatial profiles
3. Background model prediction for Background Subtraction (integrated or standalone)
4. 1/f noise removal


-----

If you use this tool in your work, please cite the tool and author(s). For questions about the tool or interested in contributing to the package in any way, please contact the authors. 

Links to Technical Reports (TBD):
1. Visit-to-visit Trace Characterization (Jdox | arXiv)
2. Visit-to-visit Wavelength Solution (Jdox | arXiv)

##  About this notebook
>**_version_**: 1.1.0
>
>**_Author_**: Tyler Baines, STScI Science Support Analyst
>
>**_Email_**: <tbaines@stsci.edu>
>
>**_Date Updated_**: 9/08/2023
>
>**_Observatory_**: JWST
>
>**_Instrument/Mode_**: NIRISS/SOSS

____